## Setup

In [1]:
reset -fs

In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

In [3]:
RANDOM_STATE = 28

## Loading Data 

In [4]:
comments = '../Data/CommentsApril2018.csv'

# Read in File
comm = pd.read_csv(comments)

/home/vietpride12/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Feature Engineering, Training, and Testing

Transforming categorical variables 'newDesk' and 'typeOfMaterial' using one-hot encoding

#### One-Hot Encoding features after removing 'Unknown' Section Names 

In [5]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
filt = (comm.sectionName != "Unknown") & (comm.commentType == "comment")
comm_filt = comm[filt].copy().reset_index()

In [6]:
# transform and map newDesk categories 
newDesk_le = LabelEncoder()
newDesk_labels = newDesk_le.fit_transform(comm_filt['newDesk'])
comm_filt['newDesk_label'] = newDesk_labels

# encoding newDesk
newDesk_ohe = OneHotEncoder()
newDesk_feature_arr = newDesk_ohe.fit_transform(comm_filt[['newDesk_label']]).toarray()
newDesk_feature_labels = list(newDesk_le.classes_)
newDesk_features = pd.DataFrame(newDesk_feature_arr, columns=newDesk_feature_labels)

In [7]:
# transform and map typeOfMaterial categories 
material_le = LabelEncoder()
material_labels = material_le.fit_transform(comm_filt['typeOfMaterial'])
comm_filt['material_label'] = material_labels

# encoding typeOfMaterial
material_ohe = OneHotEncoder()
material_feature_arr = material_ohe.fit_transform(comm_filt[['material_label']]).toarray()
material_feature_labels = list(material_le.classes_)
material_features = pd.DataFrame(material_feature_arr, columns=material_feature_labels)

In [8]:
# subsetting comment data frame and concatenating with new features
comm_filt_sub = comm_filt.loc[:,['articleID','commentID', 'commentBody', 'sectionName']]
comm_df_ohe = pd.concat([comm_filt_sub, newDesk_features, material_features], axis=1)

In [9]:
data = comm_df_ohe.loc[:,newDesk_feature_labels]
sections = comm_df_ohe.sectionName

In [10]:
train_data_no_unknown, test_data_no_unknown, train_target_no_unknown, test_target_no_unknown = train_test_split(data, sections, random_state=RANDOM_STATE)

In [12]:
lsvc_model = LinearSVC(random_state=RANDOM_STATE)
lsvc_model.fit(train_data_no_unknown, train_target_no_unknown)
lsvc_model.score(test_data_no_unknown, test_target_no_unknown)

0.7933667145902533

#### One-Hot Encoding features before removing 'Unknown' Section Names 

In [13]:
# transform and map newDesk categories 
newDesk_le = LabelEncoder()
newDesk_labels = newDesk_le.fit_transform(comm['newDesk'])
comm['newDesk_label'] = newDesk_labels

# encoding newDesk
newDesk_ohe = OneHotEncoder()
newDesk_feature_arr = newDesk_ohe.fit_transform(comm[['newDesk_label']]).toarray()
newDesk_feature_labels = list(newDesk_le.classes_)
newDesk_features = pd.DataFrame(newDesk_feature_arr, columns=newDesk_feature_labels)

In [14]:
# transform and map typeOfMaterial categories 
material_le = LabelEncoder()
material_labels = material_le.fit_transform(comm['typeOfMaterial'])
comm['material_label'] = material_labels

# encoding typeOfMaterial
material_ohe = OneHotEncoder()
material_feature_arr = material_ohe.fit_transform(comm[['material_label']]).toarray()
material_feature_labels = list(material_le.classes_)
material_features = pd.DataFrame(material_feature_arr, columns=material_feature_labels)

In [15]:
comm_sub = comm.loc[:,['articleID','commentID', 'commentBody', 'sectionName']]
comm_df_ohe = pd.concat([comm_sub, newDesk_features, material_features], axis=1)

In [16]:
data = comm_df_ohe.loc[:,newDesk_feature_labels]
sections = comm_df_ohe.sectionName

In [17]:
train_data_w_unknown, test_data_w_unknown, train_target_w_unknown, test_target_w_unknown  = train_test_split(data, sections, random_state=RANDOM_STATE)

In [19]:
lsvc_model = LinearSVC(random_state=RANDOM_STATE)
lsvc_model.fit(train_data_w_unknown, train_target_w_unknown)
lsvc_model.score(test_data_w_unknown, test_target_w_unknown)

0.7901888843592879

## Grid Search

In [20]:
from sklearn.model_selection import GridSearchCV

In [21]:
grid_params = dict(loss=['hinge', 'squared_hinge'])
lsvc_model = LinearSVC(random_state=RANDOM_STATE)

gs = GridSearchCV(estimator=lsvc_model,
                 param_grid=grid_params,
                 scoring='accuracy',
                 cv=5,
                 n_jobs=-1)

In [22]:
gs.fit(train_data_no_unknown, train_target_no_unknown)

/home/vietpride12/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=5, error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=28, tol=0.0001,
     verbose=0),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'loss': ['hinge', 'squared_hinge']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [23]:
gs.best_params_

{'loss': 'squared_hinge'}

In [24]:
gs.best_estimator_

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=28, tol=0.0001,
     verbose=0)

In [25]:
gs.score(test_data_no_unknown, test_target_no_unknown)

0.7933667145902533

<br>
<br>
<br>

## Save Model

In [26]:
from joblib import dump, load
dump(lsvc_model, '../Models/ohe_linearsvm_small.joblib') 

['../Models/ohe_linearsvm_small.joblib']

## Save Notebook

In [27]:
import dill
dill.dump_session('../Notebook_Saves/LinearSVM_OHE.db')